<a href="https://colab.research.google.com/github/aguscura/Python-Deep-Learning/blob/main/CNN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ventana capa convolucional: 5x5

Ventana capa de pooling: 2x2

Filtros: 32

Activación: ReLU.

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2)))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
Total params: 832
Trainable params: 832
Non-trainable params: 0
_________________________________________________________________


**832 parámetros.**

Como dijimos, cada filtro se compone de una ventana de (pesos + sesgo) de 5x5. En resumen, estamos agarrando una ventana de 5x5 de la imagen de entrada (de 28x28), le aplicamos este filtro mencionado y obtenemos la salida de 24x24.

Por eso la cantidad de 832 se explica así:

(5 x 5) + 1 (sesgo) = 26 parámetros para cada filtro

con 32 filtros:

parametros totales = 26 * 32 = 832.


La capa de pooling no tiene parametros entrenables porque siempre agarra el máximo.


In [10]:
# Hacemos nuestra red mas profunda, agregamos otra capa convolucional con 64 filtros mas y otra de pooling con ventana 2x2. 
# El input shape solo es necesario aclararlo en la primera capa. Luego keras lo deduce por nosotros

#Importante, ver que ahora la segunda capa convolucional tiene un tamaño de 8x8. Eso es porque la entrada de esta capa es la salida de la capa anterior (12x12).
#Luego, al tener una ventana deslizante de 5x5 y un stride o 'paso' de 1, la capa resultante es de 8x8. 

#Parámetros de la nueva capa:
#(Tamaño de la ventana * filtros de la capa anterior + sesgo ) * filtros de esta nueva capa
#       (((5x5)* 32) + 1 ) * 64 = 51264.

model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPooling2D((2,2)))

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
Total params: 52,096
Trainable params: 52,096
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Ahora sí, para hacer la clasificacion, conectamos una capa DENSAMENTE CONECTADA.
# Esta ultima capa SI VA A ESTAR CONECTADA A TODAS LAS NEURONAS DEL PASO ANTERIOR.

from tensorflow.keras import layers

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

# IMPORTANTE

Antes de agregar la ultima capa densamente conectada con la funcion de activación softmax para obtener las probabilidades, tenemos que **APLANAR** su entrada.

El tensor de entrada a una capa final softmax es de 1D, mientras que la salida que teniamos luego del segundo maxpooling era un tensor 3D (4,4,64). 

Para eso, tenemos la capa flatten que nos permite obtener un vector de [1024] ( 4x4x64).

El número de parámetros de la capa densamente conectada será: 

TAMAÑO DE LA ENTRADA * PESOS + SESGOS (siendo pesos y sesgos igual al numero de neuronas de la capa densa de salida)

 (1024 * 10) + 10 = 10.250

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 10)               